In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov3  # clone
%cd yolov3

Cloning into 'yolov3'...
remote: Enumerating objects: 9437, done.
remote: Total 9437 (delta 0), reused 0 (delta 0), pack-reused 9437
Receiving objects: 100% (9437/9437), 7.80 MiB | 19.16 MiB/s, done.
Resolving deltas: 100% (6471/6471), done.
/content/yolov3


In [ ]:
!pip install jsonpickle flask_cors flask pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.10-cp36-none-any.whl size=16377 sha256=44e745e17cd492abadbf85cdde8742cf666a0d5724718648cfd3baef98ce81f9
  Stored in directory: /root/.cache/pip/wheels/0d/7f/06/e8079dd6f80eb6683fac47c4e1cb720b61cd82d6e4c13d87a5
Successfully built pyngrok


In [ ]:
!cp  "/content/drive/My Drive/File/dataset/MedicalMaskYolo_jpg.zip" "/content"
!cp "/content/drive/My Drive/File/WeightYolov3/medicalMask_v0.2.pt" "/content/yolov3/weights"
%cd /content/
!unzip MedicalMaskYolo_jpg.zip

Streaming output truncated to the last 5000 lines.
  inflating: MedicalMaskYolo/labels/1858.txt  
  inflating: MedicalMaskYolo/labels/1825.txt  
  inflating: MedicalMaskYolo/labels/2917.txt  
  inflating: MedicalMaskYolo/labels/4420.txt  
  inflating: MedicalMaskYolo/labels/5196.txt  
  inflating: MedicalMaskYolo/labels/2367.txt  
  inflating: MedicalMaskYolo/labels/4922.txt  
  inflating: MedicalMaskYolo/labels/2925.txt  
  inflating: MedicalMaskYolo/labels/3346.txt  
  inflating: MedicalMaskYolo/labels/3243.txt  
  inflating: MedicalMaskYolo/labels/4546.txt  
  inflating: MedicalMaskYolo/labels/1905.txt  
  inflating: MedicalMaskYolo/labels/2125.txt  
  inflating: MedicalMaskYolo/labels/6295.txt  
  inflating: MedicalMaskYolo/labels/2100.txt  
  inflating: MedicalMaskYolo/labels/6222.txt  
  inflating: MedicalMaskYolo/labels/4862.txt  
  inflating: MedicalMaskYolo/labels/4579.txt  
  inflating: MedicalMaskYolo/labels/3804.txt  
  inflating: MedicalMaskYolo/labels/3495.txt  
  inflati

In [ ]:
%cd /content/yolov3
import argparse
from sys import platform

from models import * 
from utils.datasets import *
from utils.utils import *
import json

class Predict:
    def __init__(self,cfg 
                ,names 
                ,weights 
                ,imgSize    = 512
                ,confThres  = 0.3
                ,iouThres   = 0.6):
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        print("[DEVICE] "+self.device)
        self.imgSize = imgSize
        self.confThres = confThres
        self.iouThres = iouThres
        #Initialize model
        self.model = Darknet(cfg, imgSize)
        
        #load Weights
        attempt_download(weights)
        if weights.endswith('.pt'):  # pytorch format
            self.model.load_state_dict(torch.load(weights, map_location=self.device)['model'])
        else:  # darknet format
            load_darknet_weights(self.model, weights)
        self.model.to(self.device).eval()

        #load label 
        self.names = load_classes(names)
        random.seed(3)
        self.colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(self.names))]
    
    def get_predict(self,image):
        print(image.shape)
        drawing_array = np.zeros([512,512,4], dtype=np.uint8)
        img = letterbox(image, new_shape=self.imgSize)[0]
        img = img.transpose(2, 0, 1)
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(self.device)
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # (0 - 255) to (0.0 - 1.0)
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        
        pred = self.model(img)[0]
        
        pred = non_max_suppression(pred, self.confThres, self.iouThres)
        det = pred[0]

        rel = []

        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()
            
            # Write results
            for *xyxy, conf, cls in det:
                temp = {'boxes': [int(xyxy[0]),int(xyxy[1]),int(xyxy[2]),int(xyxy[3])]}
                temp["label"] = str(self.names[int(cls)])
                temp['color'] = self.colors[int(cls)]
                temp['confidence'] = round(float(conf),2)
                rel.append(temp)
                
        print("[INFO]",end= str(rel))
        print("")
        return json.dumps(rel)

/content/yolov3


In [ ]:
! ngrok authtoken 1frvcIStPu9E6VWWZUu7CZqXrqt_3owDWAZTUURPni4ja8o3v

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
from pyngrok import ngrok
import numpy as np
import argparse
import time
import cv2
import os
from flask import Flask, request, Response, jsonify
import jsonpickle
import json
#import binascii
import io as StringIO
import base64
from io import BytesIO
import io
from PIL import Image
from flask_cors import CORS
from flask import *
# construct the argument parse and parse the arguments

myModel = Predict(cfg       = "/content/MedicalMaskYolo/yolov3-mask-spp.cfg" ,
                  names     = '/content/MedicalMaskYolo/obj.names'
                  ,weights  = '/content/yolov3/weights/medicalMask_v0.2.pt')

yoloModel = Predict(cfg     = "/content/yolov3/cfg/yolov3-spp.cfg",
                    names   = "/content/yolov3/data/coco.names",
                    weights = "/content/yolov3/weights/yolov3-spp.pt")

# Initialize the Flask application
app = Flask(__name__)
CORS(app)
public_url = ngrok.connect(port = '5000')
print(public_url)
@app.route("/get-video/<video_name>")
def get_image(video_name):
  return send_from_directory("/content/",video_name, conditional=True)

@app.route('/')
def getst():
  return public_url
@app.route('/api/test', methods=['POST'])
def main():
    args = request.args
    if args:
      if 'confThres' in args:
        myModel.confThres = float(args["confThres"])
    print(myModel.confThres)

    img = request.files["image"].read()
    img = Image.open(io.BytesIO(img))
    npimg=np.array(img)
    image=npimg.copy()
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    start = time.time()
    boxs = myModel.get_predict(image)
    end = time.time()
    print("[INFO] model take: {:.6f} seconds".format(end - start))
    response = app.response_class(
        response=boxs,
        status=200,
        mimetype='application/json'
    )
    return response
    
    # start flask app
@app.route('/api/yolo', methods=['POST'])
def yoloDetect():
    img = request.files["image"].read()
    img = Image.open(io.BytesIO(img))
    npimg=np.array(img)
    image=npimg.copy()
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    start = time.time()
    boxs = yoloModel.get_predict(image)
    end = time.time()
    print("[INFO] model take: {:.6f} seconds".format(end - start))
    response = app.response_class(
        response=boxs,
        status=200,
        mimetype='application/json'
    )
    return response
print("[INFO] run app")
app.run()
